### we define elements and spaces

In [1]:
from dolfinx import fem, mesh
from dolfinx.fem.petsc import LinearProblem
from mpi4py import MPI
import ufl

element = ufl.FiniteElement("Lagrange", ufl.triangle, 1)
domain = mesh.create_unit_square(MPI.COMM_WORLD, 8, 8, mesh.CellType.triangle)

V = fem.FunctionSpace(domain, element)
x = ufl.SpatialCoordinate

# UFL dio:

In [2]:
v = ufl.TestFunction(V)
u = ufl.TrialFunction(V)

def f(x):
    return -6

a = ufl.dot(ufl.grad(v), ufl.grad(u))*ufl.dx
L = v*f(x)*ufl.dx

print(a,'\n', L)

{ (grad(v_0)) . (grad(v_1)) } * dx(<Mesh #0>[everywhere], {}) 
 { -6 * v_0 } * dx(<Mesh #0>[everywhere], {})


# FEniCSx dio:

In [3]:
uD = fem.Function(V)
uD.interpolate(lambda x: 1 + x[0]**2 + 2 * x[1]**2)

# Create facet to cell connectivity required to determine boundary facets
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)

boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
bc = fem.dirichletbc(uD, boundary_dofs)

problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [4]:
from utils import *
mh = MeshHandler()
mh.plot_function(domain, uh)

ValueError: data length of (81) != required length (128)

### Generating a mesh:

In [ ]:
# Create a mesh
mh = MeshHandler()
Omega = mh.create(N=N)

x = SpatialCoordinate
cells_H = locate_entities(Omega, Omega.topology.dim, Omega_H)
cells_T = locate_entities(Omega, Omega.topology.dim, Omega_T)

Omega_Heart, _, vertices_H, geom_H = create_submesh(Omega, Omega.topology.dim, cells_H)
Omega_Torso, _, vertices_T, geom_T = create_submesh(Omega, Omega.topology.dim, cells_T)

# mh.plot_subdomains(Omega, cells_H, cells_T)
# mh.plot_mesh(Omega)

In [ ]:
# Define function space
V2 = VectorFunctionSpace(Omega, ("CG", 2), 2)
V1 = FunctionSpace(Omega, ("CG", 2))

# Define test functions
phi, psi = TestFunctions(V2)

# Define functions
v, v_n, v_nn = Function(V2), Function(V2), Function(V2)
w, w_n, w_nn = Function(V1), Function(V1), Function(V1)
I_ion, g, I_app = Function(V1), Function(V1), Function(V1)
fibres = Function(V2)
V_m, u = v.split()
V_m_n, u_n = v_n.split()
V_m_nn, u_nn = v_nn.split()

# V_m, u = v.sub(0), v.sub(1)
# V_m_n, u_n = v_n.sub(0), v_n.sub(1)
# V_m_nn, u_nn = v_nn.sub(0), v_nn.sub(1)

### Generating a mesh:

In [ ]:
# Create a mesh
mh = MeshHandler()
Omega = mh.create(N=N)

x = SpatialCoordinate
cells_H = locate_entities(Omega, Omega.topology.dim, Omega_H)
cells_T = locate_entities(Omega, Omega.topology.dim, Omega_T)

Omega_Heart, _, vertices_H, geom_H = create_submesh(Omega, Omega.topology.dim, cells_H)
Omega_Torso, _, vertices_T, geom_T = create_submesh(Omega, Omega.topology.dim, cells_T)

# mh.plot_subdomains(Omega, cells_H, cells_T)
# mh.plot_mesh(Omega)

In [ ]:
# initial conditions
V_m_n.interpolate(lambda x: np.full((x.shape[1],), V_MIN))
w_n.interpolate(lambda x: np.full((x.shape[1],), 1 / (V_MAX - V_MIN) ** 2))
V_m_nn.interpolate(lambda x: np.full((x.shape[1],), V_MIN))
w_nn.interpolate(lambda x: np.full((x.shape[1],), 1 / (V_MAX - V_MIN) ** 2))

# interpolation part
# fibres
fun_fibres = eval_fibres()
fibres.interpolate(fun_fibres)

# ionic current
fun_I_ion = eval_I_ion(v_n, w_n)
I_ion.interpolate(fun_I_ion)

## gating variable
# fun_g = eval_g(v_n, w_n)
# g.interpolate(fun_g)

# applied current
fun_I_app = eval_I_app()
I_app.interpolate(fun_I_app)

In [ ]:
condition = lt(V_m, V_GATE)
true_statement = w / TAU_OPEN - 1 / TAU_OPEN / (V_MAX - V_MIN) ** 2
false_statement = w / TAU_CLOSE
g = conditional(condition, true_statement, false_statement)

In [ ]:
# defining UFL language parts
# conductivities
sigma_il = Constant(Omega, SIGMA_IL)
sigma_it = Constant(Omega, SIGMA_IT)
sigma_el = Constant(Omega, SIGMA_EL)
sigma_et = Constant(Omega, SIGMA_ET)
sigma_tlt = Constant(Omega, SIGMA_TLT)

d = Omega.topology.dim

sigma_i = sigma_it * Identity(d) + (sigma_il - sigma_it) * outer(fibres, fibres)
sigma_e = sigma_et * Identity(d) + (sigma_el - sigma_et) * outer(fibres, fibres)
sigma_t = sigma_tlt * Identity(d)

# Constants used in variational forms
dt = Constant(Omega, DT)
A_m = Constant(Omega, A_M)
C_m = Constant(Omega, C_M)

In [ ]:
# Define new measures associated with the interior domains and
# exterior boundaries
dx_H = Measure("dx", subdomain_data=Omega_Heart)
dx_T = Measure("dx", subdomain_data=Omega_Torso)

In [ ]:
# Define variational problem
F = (
    A_m * (C_m * (V_m - V_m_n) + dt * I_ion) * phi * dx_H
    + dt * inner(dot(sigma_i, grad(V_m + u)), grad(phi)) * dx_H
    - dt * A_m * I_app * phi * dx(1)
)
F += (
    inner(dot(sigma_i + sigma_e, grad(u)), grad(psi)) * dx_H
    + inner(dot(sigma_i, grad(V_m)), grad(psi)) * dx_H
    + inner(dot(sigma_t, grad(u)), grad(psi)) * dx_T
)

In [ ]:
print(F)

In [ ]:
from dolfinx.fem.petsc import NonlinearProblem

In [ ]:
# Time-stepping
t = 0
for n in tqdm(range(NUM_STEPS)):
    # Update current time and expressions
    t += DT
    g.v = v
    I_ion.v = v
    I_app.t = t

    # Solve variational problem for time step
    # set_log_active(False)
    problem = NonlinearProblem(F, v)
    u_ = problem.solve()

    # Update previous solution
    v_n.assign(v)